## Part 1: Data Extraction and Database Design
### Data Extraction

To extract data from the wikipedia [web-page](https://en.wikipedia.org/wiki/List_of_highest-grossing_films#Highest-grossing_films) I decided to use **requests** library to get the html content and parse it with **Beautiful soup**.

In [1]:
!pip3 install beautifulsoup4  --break-system-packages


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip


In [2]:
from bs4 import BeautifulSoup
import requests
import re
import unicodedata
import json

In [3]:
# get the content of the web-page with the table of the highest-grossing films
BASE_URL = 'https://en.wikipedia.org'
HEADERS = {'User-Agent': 'Mozilla/5.0'}
films_list_url = '/wiki/List_of_highest-grossing_films'
def get_films():
    response = requests.get(BASE_URL+films_list_url, headers=HEADERS)
    print(response.status_code)
    return BeautifulSoup(response.content, "lxml")

soup = get_films()

200


In [4]:
# extract table from the retrieved html content
table = soup.select_one('table.sortable.wikitable')
table

<table class="wikitable sortable plainrowheaders sticky-header col4right col5center col6center" style="margin:auto; margin:auto;">
<caption>Highest-grossing films<sup class="reference" id="cite_ref-12"><a href="#cite_note-12"><span class="cite-bracket">[</span>12<span class="cite-bracket">]</span></a></sup>
</caption>
<tbody><tr>
<th>Rank
</th>
<th>Peak
</th>
<th class="unsortable">Title
</th>
<th class="unsortable">Worldwide gross
</th>
<th>Year
</th>
<th class="unsortable"><abbr title="References">Ref</abbr>
</th></tr>
<tr>
<td>1
</td>
<td>1
</td>
<th scope="row"><i><a href="/wiki/Avatar_(2009_film)" title="Avatar (2009 film)">Avatar</a></i>
</th>
<td>$2,923,706,026
</td>
<td>2009
</td>
<td><sup class="reference" id="cite_ref-avatar_13-0"><a href="#cite_note-avatar-13"><span class="cite-bracket">[</span># 1<span class="cite-bracket">]</span></a></sup><sup class="reference" id="cite_ref-avatar_peak_14-0"><a href="#cite_note-avatar_peak-14"><span class="cite-bracket">[</span># 2<span c

In [5]:
# function to clean the retrieved text

def clean_text(text):
    # normalize text to avoid strange characters
    text = unicodedata.normalize("NFKD", text)
    # remove references as [5]
    text = re.sub(r'\[\s*\d+\s*\]', '', text)
    # remove whitespaces
    text = text.strip()
    return text

# extract info if data can be presented as a list
# returns 1 element if the data is not a list
def extract_list(info_text):
    # replace all <br> tags with spaces
    for br in info_text.find_all("br"):
        br.replace_with(" ")
    if info_text.find('ul'):
        info_elements = info_text.find_all('li')
        element = [clean_text(item.text) for item in info_elements]
    else:
        element_list = info_text.stripped_strings
        element = [clean_text(element_text) for element_text in element_list]
    return element

# function to get information about the films from the table
def get_films_from_table(table):
    films = []
    rows = table.select('tr')[1:]
    for row in rows:
        film_dict = {}

        attributes = row.select('td')
        # get film name
        name_field = row.select('th')[0]
        name = name_field.text.strip()
        name = clean_text(name)
        # get link to the wiki-page with detailed film info
        link = name_field.select('a')[0].attrs['href']
        # get the film rank
        rank = clean_text(attributes[0].get_text(strip=True))

        film_dict['film_name'] = name
        film_dict['link'] = link
        film_dict['rank'] = rank

        # get the year of the film
        if len(attributes) >2:
            year = attributes[3].get_text(strip=True)
        else:
            year = ""

        film_dict['year'] = year

        # from the link to the page with the detailed film info get directors, 
        # box office revenue, country and film image url
        film_url = BASE_URL + link
        response = requests.get(film_url, headers=HEADERS)
        film_soup = BeautifulSoup(response.content, "lxml")

        # just to check the content
        # with open('film_page.html', 'w', encoding='utf-8') as file:
        #     file.write(film_soup.prettify())

        # get infobox for the film
        film_infobox = film_soup.select_one('table.infobox')

        # get director
        director_row = film_infobox.find("th", string="Directed by")
        if director_row is not None:
            director_info = director_row.find_next_sibling("td")
            director = extract_list(director_info)
        else:
            director = ""

        # get box office revenue
        box_office_row = film_infobox.find("th", string="Box office")
        if box_office_row is not None:
            box_office_info = box_office_row.find_next_sibling("td")
            box_office_revenue = " ".join(box_office_info.stripped_strings)
            # remove references and clean the retrieved text
            box_office = clean_text(box_office_revenue)
        else:   
            box_office = ""

        # get country
        country = film_infobox.find("th", string=lambda text: text and "Countr" in text)
        if country is not None:
            country_info = country.find_next_sibling("td")
            country = extract_list(country_info)
            # leave only the first country
            if type(country) == list:
                country = country[0]
        else:
            country = ""

        # get the film image url
        image_tag = film_soup.find("td", class_="infobox-image")
        if image_tag is not None and image_tag.find("img"):
            image_url = image_tag.find("img")["src"]
            final_image_url = "https:" + image_url
        else:
            final_image_url = ""
        
         

        film_dict['director'] = director
        film_dict['box_office'] = box_office
        film_dict['country'] = country
        film_dict['image_url'] = final_image_url
        films.append(film_dict)

    return films

films = get_films_from_table(table)
films[0]

{'film_name': 'Avatar',
 'link': '/wiki/Avatar_(2009_film)',
 'rank': '1',
 'year': '2009',
 'director': ['James Cameron'],
 'box_office': '$2.923 billion',
 'country': 'United Kingdom',
 'image_url': 'https://upload.wikimedia.org/wikipedia/en/thumb/d/d6/Avatar_%282009_film%29_poster.jpg/220px-Avatar_%282009_film%29_poster.jpg'}

In [6]:
films[12]

{'film_name': 'Furious 7',
 'link': '/wiki/Furious_7',
 'rank': '13',
 'year': '2015',
 'director': ['James Wan'],
 'box_office': '$1.515 billion',
 'country': 'United States',
 'image_url': 'https://upload.wikimedia.org/wikipedia/en/thumb/b/b8/Furious_7_poster.jpg/220px-Furious_7_poster.jpg'}

### Database Design

I decided to store data in PostgreSQL. 

Firstly, I created database `assignment_1` using `pgadmin` application. After that, with the use of `psycopg2` library I created table, populated it with the data and then extracted all the data into the json file.

The database schema was taken from the task description, but since some films had multiple directors the director fild type was modified from `TEXT` to `TEXT[]`.

In [7]:
!pip install psycopg2-binary


In [ ]:
import psycopg2

# open connection to the database
connection = psycopg2.connect(
    dbname="assignment_1",
    user="postgres",
    # changed the password
    password="****",
    host="localhost",
    port="5432"
)
cursor = connection.cursor()

In [9]:
# if there were errors from previous runs, rollback the transaction
connection.rollback()
# create a table to store the data
create_table_query = """
    DROP TABLE IF EXISTS films;
    CREATE TABLE films (
        id SERIAL PRIMARY KEY,
        rank INTEGER,
        title TEXT NOT NULL,
        release_year INTEGER,
        director TEXT[],
        box_office TEXT,
        country TEXT,
        image_url TEXT
    );
"""
cursor.execute(create_table_query)

# populate table with the data
for film in films:
    # year is inserted as string because psycopg2 automatically converts it to integer
    insert_query = """
        INSERT INTO films (title, rank, release_year, director, box_office, country, image_url)
        VALUES (%s, %s, %s, %s, %s, %s, %s);
    """
    cursor.execute(insert_query, (film['film_name'], film['rank'], film['year'], film['director'], film['box_office'], film['country'], film['image_url']))

connection.commit()
cursor.close()
connection.close()
print('Successfully filled the table')

Successfully filled the table


### Extract data from the database to JSON file

In [ ]:
# open connection to the database to retrieve data
connection = psycopg2.connect(
    dbname="assignment_1",
    user="postgres",
    # changed the password
    password="****",
    host="localhost",
    port="5432"
)

cursor = connection.cursor()

# select all rows from the table
select_query = "SELECT * from films;"
cursor.execute(select_query)
# get all rows
data = cursor.fetchall()

column_names = []
for column in cursor.description:
    column_names.append(column[0])

print('Extracted column names:', column_names)

# store data in the list of dictionaries with film info
films_info = []
for row in data:
    cur_film_info_dict = {}
    for i, column in enumerate(column_names):
        cur_film_info_dict[column] = row[i]
    films_info.append(cur_film_info_dict)

print("Number of extracted films:", len(films_info))
print('Extracted info for the first film:', films_info[0])

cursor.close()
connection.close()

# save the data to the json file
with open('films_data.json', 'w') as file:
    json.dump(films_info, file, indent=4)


Extracted column names: ['id', 'rank', 'title', 'release_year', 'director', 'box_office', 'country', 'image_url']
Number of extracted films: 50
Extracted info for the first film: {'id': 1, 'rank': 1, 'title': 'Avatar', 'release_year': 2009, 'director': ['James Cameron'], 'box_office': '$2.923 billion', 'country': 'United Kingdom', 'image_url': 'https://upload.wikimedia.org/wikipedia/en/thumb/d/d6/Avatar_%282009_film%29_poster.jpg/220px-Avatar_%282009_film%29_poster.jpg'}
